In [ ]:
!pip install git+https://github.com/OlivierBinette/StringCompare.git@release

In [1]:
import stringcompare
import pandas as pd
import numpy as np
import wget
import zipfile
import os
from timeit import timeit

if not os.path.isfile("rawinventor.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/rawinventor.tsv.zip")
    with zipfile.ZipFile("rawinventor.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("rawinventor.tsv.zip")

In [2]:
rawinventor = pd.read_csv("rawinventor.tsv", sep="\t", usecols=["patent_id", "sequence", "name_first", "name_last"], dtype={"patent_id": "string", "sequence": "int16", "name_first": "string", "name_last": "string"})
rawinventor

,patent_id,name_first,name_last,sequence
0,6584128,Richard,Kroeger,0
1,4789863,Thomas A.,Bush,0
2,11161990,Matthew F.,Boudreaux,1
3,6795487,Gerald,Whitworth,1
4,D474886,Thomas W.,Fleming,0
...,...,...,...,...
19632552,10261193,Harald,Doppke,2
19632553,5441952,David A.,Claremon,0
19632554,4828281,David K.,Sanchas,0
19632555,10541390,Hitoshi,Yonemura,0


In [3]:
rawinventor.set_index(['patent_id', 'sequence'], inplace=True)
rawinventor.sort_index(inplace=True)

In [4]:
rawinventor

name_first name_last
patent_id sequence                      
10000000  0             Joseph    Marron
10000001  0            Sun-Woo       Lee
          1          Hyeon-Jae        Yu
10000002  0             Yun-Jo       Kim
          1             Si-Min       Kim
...                        ...       ...
T999002   0           Henry C.    Twiggs
          1           James L.     Suggs
          2             Dan F.      Buck
T999003   0            John A.      Ford
          1         Gregory J.   Lestina

[19632557 rows x 2 columns]

In [78]:
dat = rawinventor.loc['T999002']
lasts = []
first = []

dat.name_last

sequence
0    Twiggs
1     Suggs
2      Buck
Name: name_last, dtype: string

In [62]:
comparator = stringcompare.Levenshtein()

#need to figure out the best way to do this
def get_sequence(patent_id, name_first, name_last, name_middle, suffix):

    if patent_id in rawinventor.index:
        #combined names
        first_half = name_first
        second_half = name_last

        #concat middle name/initial
        if name_middle != "&":
            first_half += " " + name_middle

        #concat suffix
        if suffix != "&":
            if suffix == "2nd":
                suffix == "II"
            elif suffix == "3rd":
                suffix == "III"
            second_half += " " + suffix

        #dat = rawinventor[rawinventor.patent_id == patent_id]
        dat = rawinventor.loc[patent_id]
        last_distances = comparator.pairwise([second_half.lower()], dat.name_last.str.lower().values)[0]
        first_distances = comparator.pairwise([first_half.lower()], dat.name_first.str.lower().values)[0]

        #one last name match
        if sum(last_distances == 0) == 1: 
            return np.argmin(last_distances)

        #multiple last name matches
        elif sum(last_distances == 0) > 1:
            return np.argmin(first_distances)

        #close matches
        elif sum(last_distances < 0.2) >= 1:
            #record close data to close_match and return sequence number
            index = np.argmin(last_distances + first_distances)
            dict = {'patent_id': patent_id, 'name_last': name_last, 'name_first': name_first, 'index': index, 'table': dat}
            close_match.append(dict)
            return index
        
        #vague matches
        elif sum(last_distances < 0.3) >= 1 or sum(first_distances < 0.3) >= 1:
            #record vague data to vague_match and return sequence number
            index = np.argmin(last_distances + first_distances)
            dict = {'patent_id': patent_id, 'name_last': name_last, 'name_first': name_first, 'index': index, 'table': dat}
            vague_match.append(dict)
            return index

        #no matches
        else:
            #just get first word for each
            
            last_distances = comparator.pairwise([name_last.lower()], dat.name_last.str.lower().values)[0]
            first_distances = comparator.pairwise([name_first.lower()], dat.name_first.str.lower().values)[0]


            #still record data but return "NaN"
            index = np.argmin(last_distances + first_distances)
            dict = {'patent_id': patent_id, 'name_last': name_last, 'name_first': name_first, 'index': index, 'table': dat}
            no_match.append(dict)
            return "NaN"
    else:
        no_key.append(patent_id)
        return "NaN"
        

In [64]:
#lai_benchmark = pd.read_excel("benchmark.xlsx", dtype=str).iloc[3:, :]
lai_benchmark = pd.read_csv("patents_2005_012.tsv", sep="\t", usecols=["patent", "fname", "mname", "lname", "suffix"], dtype="string").sample(1000)
lai_benchmark.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 26648 to 41942
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   patent  1000 non-null   string
 1   fname   1000 non-null   string
 2   mname   1000 non-null   string
 3   lname   1000 non-null   string
 4   suffix  1000 non-null   string
dtypes: string(5)
memory usage: 46.9 KB


In [ ]:
no_key = []
close_match = []
vague_match = []
no_match = []
lai_benchmark["sequence"] = lai_benchmark.apply(lambda x: get_sequence(x.patent, x.fname, x.lname, x.mname, x.suffix), axis=1)

In [63]:
lai_benchmark.to_csv("patents_2005_012_autosequence.csv")

In [57]:
#look at results
print("Close Match Size:", len(close_match))
print("Vague Match Size", len(vague_match))
print("No Match Size:", len(no_match))
print("No Key Size:", len(no_key))

Close Match Size: 13
Vague Match Size 5
No Match Size: 9
No Key Size: 0


In [61]:
no_match[1]

{'patent_id': '5825448',
 'name_last': 'Glenn Jr.',
 'name_first': 'William E.',
 'index': 3,
 'table':           name_first          name_last
 sequence                               
 0          Philip J.                Bos
 1            Jianmin               Chen
 2           David L.  Johnson, deceased
 3         William E.              Glenn
 4          Carvel E.             Holton
 5           Brett E.              Smith}